In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
data = pd.read_csv('accepted_2007_to_2018Q4.csv',parse_dates=['issue_d'], infer_datetime_format=True)
data = data.reset_index(drop=True)
data.head()

C:\Users\shiva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
columns = ["annual_inc", "delinq_2yrs", "dti", "emp_length",
           "grade", "home_ownership", "installment", "int_rate", 
           "loan_amnt", "loan_status", "num_accts_ever_120_pd", 
           "purpose", "term", "total_acc", "total_pymnt", 
           "fico_range_high", "fico_range_low"]
data=data[columns]

In [4]:
data['term'] = data['term'].str.split(' ').str.get(1)
data['term'] = data['term'].astype('float')
data['int_rate'] = pd.to_numeric(data['int_rate'])
data['loan_status']=data['loan_status'].astype('category')
data['emp_length'] = data['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
data['emp_length'] = data['emp_length'].str.extract('(\d+)').astype('float')
data['emp_length']

0          11.0
1          11.0
2          11.0
3          11.0
4           3.0
           ... 
2260696     9.0
2260697     6.0
2260698    11.0
2260699     NaN
2260700     NaN
Name: emp_length, Length: 2260701, dtype: float64

In [5]:
data=data.dropna()

In [6]:
data["expected_payment"] = data["installment"] * data["term"]
data["recovery_rate"] = data["total_pymnt"] / data["expected_payment"] * 100
data.drop(['installment','term'],axis=1)
data.head()

,annual_inc,delinq_2yrs,dti,emp_length,grade,home_ownership,installment,int_rate,loan_amnt,loan_status,num_accts_ever_120_pd,purpose,term,total_acc,total_pymnt,fico_range_high,fico_range_low,expected_payment,recovery_rate
0,55000.0,0.0,5.91,11.0,C,MORTGAGE,123.03,13.99,3600.0,Fully Paid,2.0,debt_consolidation,36.0,13.0,4421.723917,679.0,675.0,4429.08,99.833914
1,65000.0,1.0,16.06,11.0,C,MORTGAGE,820.28,11.99,24700.0,Fully Paid,0.0,small_business,36.0,38.0,25679.660000,719.0,715.0,29530.08,86.961024
2,63000.0,0.0,10.78,11.0,B,MORTGAGE,432.66,10.78,20000.0,Fully Paid,0.0,home_improvement,60.0,18.0,22705.924294,699.0,695.0,25959.60,87.466387
3,110000.0,0.0,17.06,11.0,C,MORTGAGE,829.90,14.85,35000.0,Current,0.0,debt_consolidation,60.0,17.0,31464.010000,789.0,785.0,49794.00,63.188356
4,104433.0,1.0,25.37,3.0,F,MORTGAGE,289.91,22.45,10400.0,Fully Paid,0.0,major_purchase,60.0,35.0,11740.500000,699.0,695.0,17394.60,67.495085


In [7]:
data=data[data['loan_status']!='Current']

In [8]:
categorical = ["grade", "home_ownership", "purpose"]
data = pd.get_dummies(data, columns=categorical, prefix="", prefix_sep="", drop_first=True)

In [10]:
data[data['loan_status']!='Fully Paid']['recovery_rate'].mean()

40.96232006210536

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data.drop(['recovery_rate','loan_status'],axis=1))
x=pd.DataFrame(scaler.transform(data.drop(['recovery_rate','loan_status'],axis=1)))

In [12]:
#x=data.drop(['recovery_rate','loan_status'],axis=1)
y=data['recovery_rate']

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [14]:
from sklearn.linear_model import LinearRegression
lreg = LinearRegression()
lreg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
lreg.coef_

array([-1.74978334e+01,  1.12112714e+00, -3.25684261e+01,  2.45673470e-01,
       -2.09983115e+01, -3.09700436e+01, -1.29862319e+02,  1.50610372e+00,
       -1.40928854e+01, -7.00547539e-01,  2.10643097e+02,  2.91056793e+00,
       -2.73110885e+00,  1.99735855e+01, -9.89779638e-03, -8.25566523e-01,
       -1.03046777e+00, -1.37417765e+00, -5.72979380e-01,  1.57925747e-01,
        3.23003046e+00,  4.39568564e+00,  6.26891459e+00,  2.92121634e+00,
        2.75804637e+00, -3.79475054e-01, -5.20042135e-01, -7.53175300e-13,
       -4.57916151e-01, -1.15399370e+00,  4.54505503e-02, -1.40993215e+00,
       -1.05828555e+00, -4.48084989e-01, -1.56068041e+00, -4.16114445e-01,
       -5.36573418e-01,  4.10974888e+00])

In [16]:
test=lreg.predict(x_test)

In [17]:
test

array([51.65756625, 75.57554849, 81.63937874, ..., 99.41106134,
       93.96319631, 47.29881071])

In [18]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,test)

231.0109814179879

In [19]:
from sklearn.linear_model import Ridge
alpha_range = 10.**np.arange(-2, 3)
alpha_range
# select the best alpha with RidgeCV
from sklearn.linear_model import RidgeCV
ridgeregcv = RidgeCV(alphas=alpha_range, scoring='neg_mean_squared_error')
ridgeregcv.fit(x_train, y_train)
ridgeregcv.alpha_

0.1

In [20]:
from sklearn.linear_model import Ridge
ridgereg = Ridge(alpha=0.1)
ridgereg.fit(x_train, y_train)
ry_pred = ridgereg.predict(x_test)
mean_squared_error(y_test,ry_pred)

231.0105863332528

In [21]:
ridgereg.coef_

array([-1.74326689e+01,  1.12122656e+00, -3.25165445e+01,  2.45695394e-01,
       -2.10251323e+01, -3.09612299e+01, -1.29815208e+02,  1.50731880e+00,
       -1.40941297e+01, -7.02716616e-01,  2.10638452e+02,  1.21658319e-01,
        4.24677077e-02,  1.99336926e+01, -1.04811716e-02, -8.26630200e-01,
       -1.03173673e+00, -1.37506953e+00, -5.73038268e-01,  1.58730709e-01,
        3.22035072e+00,  4.37250055e+00,  6.23927908e+00,  2.91151774e+00,
        2.74838636e+00, -3.79640076e-01, -5.20184488e-01,  0.00000000e+00,
       -4.57997323e-01, -1.15424443e+00,  4.54209695e-02, -1.41020983e+00,
       -1.05876831e+00, -4.48395781e-01, -1.56080982e+00, -4.16346975e-01,
       -5.36889099e-01,  4.10862977e+00])

In [22]:
from sklearn.linear_model import LassoCV
lassoregcv = LassoCV(n_alphas=100, random_state=1)
lassoregcv.fit(x_train, y_train)
print('alpha : ',lassoregcv.alpha_)

alpha :  0.0038008929259824645


In [23]:
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.0038008929259824645)
lassoreg.fit(x_train, y_train)
ly_pred = lassoreg.predict(x_test)
mean_squared_error(y_test,ly_pred)

231.11200108760403

In [24]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.88)
x_1=pd.DataFrame(pca.fit_transform(x))

In [25]:
x_1

,0,1,2,3,4,5,6,7,8,9
0,0.577782,-0.489373,0.193936,-0.888871,0.146928,-0.005004,-0.357694,0.000248,-0.241994,-0.316150
1,0.684767,0.138626,0.646663,-0.569826,0.002152,0.054600,-0.362529,0.123740,0.232768,0.463067
2,0.864357,0.510001,-0.200936,0.486771,-0.541946,0.184595,-0.188428,0.920362,-0.233082,0.432163
3,0.648629,0.118471,0.437546,0.544288,-0.036221,-0.196766,0.418613,0.643794,-0.012851,-0.224886
4,-0.831537,-0.621634,0.256382,-0.626889,-0.080863,-0.162971,0.023272,-0.086067,0.011034,0.081700
...,...,...,...,...,...,...,...,...,...,...
1232676,0.071859,0.518988,-0.183664,0.544019,-0.480209,1.140705,0.660299,0.754267,-0.162648,0.499689
1232677,0.839966,-0.828543,0.567116,-0.051911,-0.635950,-0.010118,-0.100017,-0.059469,-0.088354,0.248638
1232678,0.782397,-0.041990,0.986636,0.005484,-0.401613,-0.099970,0.444028,0.721078,0.045018,0.687608
1232679,-0.608546,-0.190132,0.953032,0.066288,-0.527953,0.149522,-0.466407,0.545378,-0.275395,0.137683


In [26]:
from sklearn.preprocessing import PolynomialFeatures 
poly = PolynomialFeatures(degree = 2) 
x_poly = poly.fit_transform(x_1)
x_poly=pd.DataFrame(x_poly)

In [27]:
x_poly

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,1.0,0.577782,-0.489373,0.193936,-0.888871,0.146928,-0.005004,-0.357694,0.000248,-0.241994,...,0.127945,-0.000089,0.086560,0.113085,6.148468e-08,-0.000060,-0.000078,0.058561,0.076506,0.099951
1,1.0,0.684767,0.138626,0.646663,-0.569826,0.002152,0.054600,-0.362529,0.123740,0.232768,...,0.131427,-0.044859,-0.084385,-0.167875,1.531152e-02,0.028803,0.057300,0.054181,0.107787,0.214431
2,1.0,0.864357,0.510001,-0.200936,0.486771,-0.541946,0.184595,-0.188428,0.920362,-0.233082,...,0.035505,-0.173422,0.043919,-0.081432,8.470656e-01,-0.214519,0.397746,0.054327,-0.100729,0.186765
3,1.0,0.648629,0.118471,0.437546,0.544288,-0.036221,-0.196766,0.418613,0.643794,-0.012851,...,0.175237,0.269501,-0.005380,-0.094140,4.144712e-01,-0.008273,-0.144780,0.000165,0.002890,0.050574
4,1.0,-0.831537,-0.621634,0.256382,-0.626889,-0.080863,-0.162971,0.023272,-0.086067,0.011034,...,0.000542,-0.002003,0.000257,0.001901,7.407518e-03,-0.000950,-0.007032,0.000122,0.000902,0.006675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232676,1.0,0.071859,0.518988,-0.183664,0.544019,-0.480209,1.140705,0.660299,0.754267,-0.162648,...,0.435995,0.498042,-0.107396,0.329944,5.689188e-01,-0.122680,0.376899,0.026454,-0.081273,0.249689
1232677,1.0,0.839966,-0.828543,0.567116,-0.051911,-0.635950,-0.010118,-0.100017,-0.059469,-0.088354,...,0.010003,0.005948,0.008837,-0.024868,3.536566e-03,0.005254,-0.014786,0.007806,-0.021968,0.061821
1232678,1.0,0.782397,-0.041990,0.986636,0.005484,-0.401613,-0.099970,0.444028,0.721078,0.045018,...,0.197161,0.320179,0.019989,0.305317,5.199531e-01,0.032461,0.495818,0.002027,0.030955,0.472804
1232679,1.0,-0.608546,-0.190132,0.953032,0.066288,-0.527953,0.149522,-0.466407,0.545378,-0.275395,...,0.217536,-0.254368,0.128446,-0.064216,2.974368e-01,-0.150194,0.075089,0.075843,-0.037917,0.018957


In [28]:
from sklearn.model_selection import train_test_split
px_train, px_test, py_train, py_test = train_test_split(x_poly, y, test_size=0.33, random_state=42)
preg = LinearRegression()
preg.fit(px_train,py_train)
p_test=preg.predict(px_test)
mean_squared_error(py_test,p_test)

575.1138051140383